In [ ]:
# 6类数据，分为浅呼吸躺，深呼吸躺，浅呼吸坐，深呼吸坐，浅呼吸走，深呼吸走， 0代表胸式呼吸 1代表腹式呼吸

In [ ]:
# 统计一下data.xlsx每一个label分别有多少个数据，总共有6个label（呼吸躺，深呼吸躺，浅呼吸坐，深呼吸坐，浅呼吸走，深呼吸走， 0代表胸式呼吸 1代表腹式呼吸）

In [2]:
import pandas as pd
import torch
import torch.nn as nn

# 读取Excel文件
file_path = 'data/data.xlsx'
df = pd.read_excel(file_path, sheet_name='Sheet1',header=None)

# 统计每个label的数量
label_counts = df.iloc[:,-1].value_counts()

# 打印统计结果
print(label_counts)


6
lying-1    17625
sit-0      16103
lying-0    15824
sit-1      14176
walk-1     10012
walk-0      7519
label          1
Name: count, dtype: int64


In [3]:
import pandas as pd
df = pd.read_excel(file_path, sheet_name='Sheet1')
label_counts = df['label'].value_counts()
print(label_counts)

label
lying-1    17625
sit-0      16103
lying-0    15824
sit-1      14176
walk-1     10012
walk-0      7519
Name: count, dtype: int64


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 提取特征和标签
features = df[['x_imu', 'y_imu', 'z_imu', 'x_gry', 'y_gry', 'z_gry']]
labels = df['label']

# 标签编码
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)

# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# 转换为PyTorch张量
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [9]:
from torch.utils.data import DataLoader, TensorDataset

# 创建数据集和数据加载器
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)



In [27]:
import torch.nn as nn
import torch.optim as optim

class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        # RNN层
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        # 全连接层
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

# 参数
input_size = features.shape[1]  # 特征数量
hidden_size = 64
output_size = len(label_encoder.classes_)  # 标签类别数量
num_layers = 1

# 实例化模型、损失函数和优化器
model = RNNModel(input_size, hidden_size, output_size, num_layers)
criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [28]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for i, (inputs, labels) in enumerate(train_loader):
        outputs = model(inputs.unsqueeze(1))  # 需要增加一个维度以适应 RNN 的输入
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')


Epoch [1/10], Step [10/4063], Loss: 1.7152
Epoch [1/10], Step [20/4063], Loss: 1.6273
Epoch [1/10], Step [30/4063], Loss: 1.5584
Epoch [1/10], Step [40/4063], Loss: 1.5026
Epoch [1/10], Step [50/4063], Loss: 1.3251
Epoch [1/10], Step [60/4063], Loss: 1.6033
Epoch [1/10], Step [70/4063], Loss: 1.4645
Epoch [1/10], Step [80/4063], Loss: 1.3668
Epoch [1/10], Step [90/4063], Loss: 1.5445
Epoch [1/10], Step [100/4063], Loss: 1.3998
Epoch [1/10], Step [110/4063], Loss: 1.4043
Epoch [1/10], Step [120/4063], Loss: 1.4805
Epoch [1/10], Step [130/4063], Loss: 1.4455
Epoch [1/10], Step [140/4063], Loss: 1.4682
Epoch [1/10], Step [150/4063], Loss: 1.2368
Epoch [1/10], Step [160/4063], Loss: 0.9514
Epoch [1/10], Step [170/4063], Loss: 1.1947
Epoch [1/10], Step [180/4063], Loss: 1.4369
Epoch [1/10], Step [190/4063], Loss: 1.2664
Epoch [1/10], Step [200/4063], Loss: 1.5006
Epoch [1/10], Step [210/4063], Loss: 1.2934
Epoch [1/10], Step [220/4063], Loss: 1.3213
Epoch [1/10], Step [230/4063], Loss: 1.58

第一次的学习：学习率
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [14]:
from sklearn.metrics import confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 生成混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)


[[2836  315    3    2    0    0]
 [ 328 3128   22    7    0    0]
 [   0    1 3059   28   23  126]
 [   0    0    5 2705    2  169]
 [   1   13  673   22  126  668]
 [   0    2  640    1  100 1247]]


第二次的学习：学习率
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [17]:
from sklearn.metrics import confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 生成混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)

[[2838    0    0  294    0   24]
 [ 328 2171    0  923    0   63]
 [  23    0    0 3116    0   98]
 [   2    0    0 2709    0  170]
 [ 115    2   24  731    0  631]
 [ 104    0    0  729    0 1157]]


第三次的学习：学习率
optimizer = optim.Adam(model.parameters(), lr=0.0005) weight_decay=1e-5

In [20]:
from sklearn.metrics import confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 生成混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)

[[2852  301    3    0    0    0]
 [ 339 3096   22    0    0   28]
 [   1    0 3071   47    0  118]
 [   0    0   68 2683    3  127]
 [   7   13  730   19    1  733]
 [   1    4  729   33    0 1223]]


增加惩罚系数在lr最好的

In [23]:
from sklearn.metrics import confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 生成混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)


[[2857  239    2   58    0    0]
 [ 341 2685    0  459    0    0]
 [   1    3 3081    9    0  143]
 [   0   87  644 2127    1   22]
 [   4   68  705   16    2  708]
 [   2  206  641    3    0 1138]]


增加惩罚项之后效果很差

In [26]:
from sklearn.metrics import confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 生成混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)


[[   2    0 3154    0    0    0]
 [   0 2192 1249    0    0   44]
 [   0    5 3226    0    0    6]
 [   0   47 2788    0    0   46]
 [   4  230 1209    0    0   60]
 [   0  480 1355    0    0  155]]


感觉仅仅学习率0.001的时候稍微好一点点

In [29]:
from sklearn.metrics import confusion_matrix

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs.unsqueeze(1))
        _, predicted = torch.max(outputs.data, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# 生成混淆矩阵
conf_matrix = confusion_matrix(all_labels, all_preds)
print(conf_matrix)


[[2850   62  241    0    1    2]
 [ 339 2668  471    7    0    0]
 [   0    1 3116    0   23   97]
 [   0    0 2709    0    5  167]
 [   0   17  743    2  107  634]
 [   0    7  733    0  100 1150]]


In [ ]:
import serial
import numpy as np

#   ESP32-C3 使用的波特率为 115200
ser = serial.Serial('/dev/ttyACM0', 115200, timeout=1)

def read_imu_data():
    data = []
    while len(data) < 200:  # 读取200个数据点
        line = ser.readline().decode('utf-8').strip()
        if line:
            try:
                values = list(map(float, line.split(',')))  # 假设IMU数据以逗号分隔
                if len(values) == 6:  # 确保数据格式正确（3个IMU + 3个陀螺仪数据）
                    data.append(values)
            except ValueError:
                pass  # 跳过不正确的行
    return np.array(data)

# 读取IMU数据
imu_data = read_imu_data()


In [ ]:
import torch

# 假设数据格式是 [x_imu, y_imu, z_imu, x_gry, y_gry, z_gry]
def prepare_data(imu_data):
    # 将数据转换为PyTorch张量
    imu_tensor = torch.tensor(imu_data, dtype=torch.float32)
    # 添加批次维度和时间步维度，使得输入符合RNN的要求
    imu_tensor = imu_tensor.unsqueeze(0).unsqueeze(1)  # (batch_size, seq_length, feature_size)
    return imu_tensor

# 准备数据
input_tensor = prepare_data(imu_data)


In [ ]:
import torch

# 假设数据格式是 [x_imu, y_imu, z_imu, x_gry, y_gry, z_gry]
def prepare_data(imu_data):
    # 将数据转换为PyTorch张量
    imu_tensor = torch.tensor(imu_data, dtype=torch.float32)
    # 添加批次维度和时间步维度，使得输入符合RNN的要求
    imu_tensor = imu_tensor.unsqueeze(0).unsqueeze(1)  # (batch_size, seq_length, feature_size)
    return imu_tensor

# 准备数据
input_tensor = prepare_data(imu_data)


In [ ]:
model = RNNModel(input_size=6, hidden_size=64, output_size=len(label_encoder.classes_), num_layers=1)
model.load_state_dict(torch.load('rnn_model.pth'))
model.eval()

# 使用模型进行推理
with torch.no_grad():
    output = model(input_tensor)
    _, predicted_label = torch.max(output, 1)

# 将预测结果转换为实际标签
predicted_label_str = label_encoder.inverse_transform(predicted_label.cpu().numpy())
print(f'Predicted label: {predicted_label_str[0]}')


In [ ]:
import time

while True:
    imu_data = read_imu_data()
    input_tensor = prepare_data(imu_data)
    
    with torch.no_grad():
        output = model(input_tensor)
        _, predicted_label = torch.max(output, 1)
    
    predicted_label_str = label_encoder.inverse_transform(predicted_label.cpu().numpy())
    print(f'Predicted label: {predicted_label_str[0]}')

    # 可以加一个暂停时间，避免无限循环
    time.sleep(1)  # 例如，每秒推理一次
